In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam 
from model.resnet import ResNet
from utils.data_config import Config_Cat_4
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
from tensorflow.keras.models import Sequential, Model

data_config = Config_Cat_4()

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

for gpu in physical_devices:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [3]:
# define the total number of epochs to train for along with the
# initial learning rate and batch size
NUM_EPOCHS = 20
INIT_LR = 1e-1
BS = 32

In [4]:
def poly_decay(epoch):
	# initialize the maximum number of epochs, base learning rate,
	# and power of the polynomial
	maxEpochs = NUM_EPOCHS
	baseLR = INIT_LR
	power = 1.0

	# compute the new learning rate based on polynomial decay
	alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power

	# return the new learning rate
	return alpha

In [5]:
# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(data_config.training_path)))
totalVal = len(list(paths.list_images(data_config.val_path)))
totalTest = len(list(paths.list_images(data_config.test_path)))

In [6]:
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.05,
	height_shift_range=0.05,
	shear_range=0.05,
	horizontal_flip=True,
	fill_mode="nearest")

In [7]:
valAug = ImageDataGenerator(rescale=1 / 255.0)

In [8]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	data_config.training_path,
	class_mode="binary",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	data_config.val_path,
	class_mode="binary",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 412 images belonging to 2 classes.
Found 203 images belonging to 2 classes.


In [9]:
# initialize the testing generator
testGen = valAug.flow_from_directory(
	data_config.test_path,
	class_mode="binary",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 395 images belonging to 2 classes.


In [ ]:
# initialize our ResNet model and compile it
model = ResNet.build(64, 64, 3, 2, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
opt = SGD(lr=INIT_LR, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
model.reset_metrics()

In [ ]:
# define our set of callbacks and fit the model
callbacks = [LearningRateScheduler(poly_decay)]
H = model.fit(
	trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	epochs=NUM_EPOCHS,
	callbacks=callbacks)

In [ ]:
# reset the testing generator and then use our trained model to
# make predictions on the data
model.save('ResNet_Category4_model.h5')

print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(testGen, steps=(totalTest // BS) + 1)
 
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.where(predIdxs>0.5,1,0)
print()

# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))

# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_category4_resnet")

In [10]:
model_saved = tf.keras.models.load_model('model/saved_models/ResNet_Category4_model.h5')
model= Model(inputs=model_saved.input, outputs=model_saved.layers[-3].output)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   4800        batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________

In [11]:
print("[INFO] evaluating after initialization...")
predTest = model.predict(testGen, steps=(totalTest // BS) + 1)

print(predTest.shape)
np.savetxt('point-4_resnet_test_features.csv', predTest, delimiter=',')

[INFO] evaluating after initialization...
(395, 512)


In [12]:
print("[INFO] evaluating after initialization...")
predVal = model.predict(valGen, steps=(totalVal // BS) + 1)

print(predVal.shape)
np.savetxt('point-4_resnet_val_features.csv', predVal, delimiter=',')

[INFO] evaluating after initialization...
(203, 512)
